<a href="https://colab.research.google.com/github/dtabuena/patchclamp_analysis/blob/main/Testing_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall patchclamp_analysis -y
!pip install https://github.com/dtabuena/patchclamp_analysis/archive/main.zip -q --upgrade
from patchclamp_analysis import (
    ephys_utilities,
    abf_handling,
    analysis_parameters,
    capacitance_analysis,
    firing_gain_analysis,
    input_resistance_analyzer,
    iv_analysis,
    resting_potential_analysis,
    rheobase_analaysis,
    spike_latency_analysis,
    pipeline_functions
)

In [ ]:
from patchclamp_analysis.pipeline_functions import (analysis_iterator_h5,
                                                    build_analysis_h5,
                                                    print_h5_tree,
                                                    restratify_h5_by_attribute,
                                                    add_current_density_datasets,
                                                    get_datasets_from_h5,
                                                    flatten_nested_column,
                                                    write_dict_to_excel)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from IPython.display import clear_output
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import pyabf
import h5py
import urllib

working_dir = r'\\hive.gladstone.internal\Huang-Lab\Lab Members\Tabuena_Main\0_Projects_hv\testing_h5'
os.chdir(working_dir)

response = urllib.request.urlretrieve('https://raw.githubusercontent.com/dtabuena/Resources/main/Matplotlib_Config/Load_FS6.py','Load_FS6.py')
%run Load_FS6.py

In [ ]:

dataset_info = {'data_name': 'MAPT_DLX_SST',
           'data_source': os.path.join(working_dir),
           'file_naming_scheme': ['rec_date','genotype','sex','age', 'orientation','slice','cell_num','cell_type'],
           }


##### Setup Protocol List
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',
           'VC - Multi IV - 500ms']
IC_prot = ['IC - Gain - D10pA',
           'IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree',
           'I0 - 3min GapFree']

analysis_dir =os.path.join(working_dir,dataset_info['data_source'])
os.makedirs(analysis_dir,exist_ok=True)
os.chdir(analysis_dir)
print(os.getcwd())


In [ ]:
"""
Import And Catalog ABFS
"""

h5_data_loc = build_analysis_h5(dataset_info)
print_h5_tree(h5_data_loc, head=10)

In [ ]:
"""
Analyze All Recordings
"""
os.chdir(working_dir)
analyzer_configs = analysis_parameters.init_func_arg_dicts_h5()
analysis_iterator_h5(h5_data_loc,analyzer_configs)

In [ ]:
"""
START HERE TO SKIP ANALYSIS
"""
h5_data_loc = 'MAPT_DLX_SST_analysis_recs.h5'

In [ ]:
"""
Convert Recording Groups to Cell Groups
"""
cell_h5_loc = restratify_h5_by_attribute(h5_data_loc, 'MAPT_DLX_SST_analysis_cells.h5', 'cell_id')

In [ ]:
"""
Add Current Densisty
"""
failed_groups = add_current_density_datasets(cell_h5_loc)
print_h5_tree(cell_h5_loc,head=45)

In [ ]:
"""
Sort and Export to CSV
"""

data_key = 'density_IV_Na_0.2_10'
# Multiple datasets with multiple attributes
df = get_datasets_from_h5(cell_h5_loc,
                          [data_key],
                          ['genotype'])

result_dfs  = flatten_nested_column(df,'genotype',data_key)
for k,v in result_dfs.items():
    print(k)
write_dict_to_excel(result_dfs, data_key+'.xlsx')